# Form

## Setup

In [1]:
#| default_exp form

In [3]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *

from fh_commons.core import *
from fh_commons.static import *

import json
import pandas as pd
from math import ceil

In [4]:
hdrs = bootstrap_hdrs()+download_js()+datatable_hdrs()

In [5]:
show(*hdrs)

In [6]:
#| eval: False
from ngrok_token import *
url = start_ngrok(token)

ngrok tunnel opened at: https://a223-3-238-95-91.ngrok-free.app


In [7]:
app,rt = fast_app(pico=False,hdrs=hdrs)
server = JupyUvi(app)

## File upload & receive

### Upload csv/excel file 

In [32]:
#| export
def get_file_input(label_text,id,**kwargs):
    return Form(
        Label(label_text, fr=id, cls='form-label'),
        Input(type='file', id=id, cls='form-control'),
        **kwargs
    )

In [33]:
%%s
get_file_input('Select csv/excel for upload',id='sdf')

### Receive file

In [34]:
#| export
def bytes2df(bytes_data,file_type):
    if file_type == "csv":
        data_io = StringIO(bytes_data.decode("utf-8"))
        return pd.read_csv(data_io)
    elif file_type == "excel":
        data_io = BytesIO(bytes_data)
        return pd.read_excel(data_io)

### Example

In [35]:
@rt
def file_upload(req):
    add = get_file_input(
        'Upload your csv or excel',
        id='myFile',post=preview, target_id='content',hx_trigger='change')
    
    example_file = A(Small('Download Example File'), href=req.url_for('download_example'),download='example.csv')
    
    return Group(add,example_file), Div(id='content')

@app.get
def download_example():
    return FileResponse('data/example.csv')

@app.post
async def preview(sess, myFile:UploadFile):
    filename = myFile.filename

    if filename.endswith('.csv'):
        file_type = 'csv'
    elif filename.endswith(('.xls', '.xlsx')):
        file_type = 'excel'
    else:
        return add_toast(sess, "Please upload a csv or excel file", "error")
        
    # Data as byte string
    bytes_data = await myFile.read()
    df = bytes2df(bytes_data,file_type)
    
    if len(df)>100_000:
        return add_toast(sess, "Exceed 100,000 lines, please use python api for large file", "warning")
    
    return df2html(df.head())

In [36]:
htmx(url,'/file_upload')

## Input text

In [11]:
#| export
def get_input(label_text,id,cls=None, type='text',**kwargs):
    return Span(
        Input(type=type, id=id, cls='form-control',placeholder=label_text,**kwargs),
        Label(label_text, fr=id),
        cls=f'form-floating {cls}')

In [12]:
%%s
get_input('Gene ID',id='aa')

In [16]:
#| export
def get_button(text,cls,**kwargs):
    return Button(text,cls=f"btn btn-primary {cls}",type='button',**kwargs)

In [21]:
%%s
Form(get_input('sdf','a',cls='col-3'),get_button('aaa',cls='col-1'),cls='row g-2')

In [62]:
#| export
def get_input_list(label_text, input_list, id, cls=None, type='text', **kwargs):
    input_div = Div(
        Input(type=type, id=id, cls='form-control', placeholder=label_text, autocomplete="off", **kwargs),
        Label(label_text, fr=id),
        Div(
            *[Div(item, cls='dropdown-item', data_value=item) for item in input_list],
            id=f'{id}-dropdown',
            cls='dropdown-menu'
        ),
        cls=f'form-floating position-relative {cls}'
    )

    script = Script(f"""
    document.addEventListener('DOMContentLoaded', function() {{
        setupAutocomplete('#{id}', '#{id}-dropdown');
    }});
    """)

    return input_div, script

In [64]:
%%s
get_input_list('Select item', ['a','b','c','d'],id='select')

## TextArea

In [65]:
%%s
Textarea(id='s', placeholder='sdf', cls='form-control',rows=10)

## Select (default is first)

In [66]:
#| export
def get_select(label_text,option_list,id,cls=None, **kwargs):
    "The first item in the option_list is the default"
    return Div(
        Select(
            Option(option_list[0], value=option_list[0],selected=True),
            *[Option(i, value=i) for i in option_list[1:]],
            id = id,
            cls='form-select',
            **kwargs
        ),
        Label(label_text, fr=id),
        cls=f'form-floating {cls}')


## Select

In [67]:
#| export
def get_select_simple(label_text, select_list,id,cls=None):
    return Div(Select(
        Option(label_text, selected=True,disabled=True),
        *[Option(i,value=i) for i in select_list],
        cls='form-select',
        id = id), cls=cls)

In [68]:
%%s
get_select_simple('select one', ['a','b','c'],id='ssdf')

## Select multiple

In [71]:
#| export
def get_select_simple_multiple(label_text, select_list,id,cls=None):
    return Div(Select(
        Option(label_text, selected=True,disabled=True),
        *[Option(i,value=i) for i in select_list],
        cls='form-select',
        multiple=True,
        id = id), cls=cls)

In [72]:
%%s
get_select_simple_multiple('select one', ['a','b','c'],id='ssss')

## Spinner

In [73]:
#| export
def get_spinner(id, cls='d-flex justify-content-center'):
    return Div(Div(
        Span('Loading...', cls='visually-hidden'),
        role='status',
        cls='htmx-indicator spinner-border',
        id=id,
    ),cls=cls)

In [75]:
%%s
get_spinner(id='sdf')

Example:

In [74]:
# @app.post
# def example():
#     ...
#     send_select = Form(
#             Hidden(value=df.to_json(),id='upload_df'),
#             Div(select_list,button,cls='row g-2 align-items-center'),
#             post=calculate,
#             target_id='result',
#             hx_indicator='#spinner',
#         )
    
#     spinner=get_spinner(id='spinner')
    
#     return send_select, spinner, ...


## End

In [29]:
#| hide
import nbdev; nbdev.nbdev_export()

In [78]:
server.stop()
kill_ngrok()

ngrok tunnel killed
